In [24]:
import re

symbol = 'BTC3SUP'
re.match(r'.+[23][LS].+', symbol)
# re.match(r'.?DOWN', '', symbol)

# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 3, 10, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:10<00:00,  3.53s/it]


# Check local statistics

In [16]:
# stat = pd.read_pickle('opt_PriceChange_buy_5m_1h.pkl')
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('forecast_rank', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
0,PriceChange,1,100.00,50.00,62.50,50.00,62.50,37.50,50.00,62.50,50.00,62.50,62.50,62.50,62.50,62.50,50.00,50.00,50.00,50.00,50.00,50.00,50.00,62.50,62.50,62.50,0.66,-0.17,0.32,0.21,0.17,-0.04,0.07,1.49,0.67,1.49,1.13,2.00,1.34,1.04,0.05,-0.23,-0.04,0.10,-0.18,0.10,-0.08,0.65,2.21,2.22,8,57.291667,0.632500,-101.666667,5.060000
2,PriceChange,3,100.00,73.08,73.08,65.38,69.23,65.38,57.69,69.23,57.69,65.38,65.38,61.54,61.54,65.38,57.69,65.38,57.69,53.85,61.54,50.00,50.00,50.00,46.15,50.00,0.75,1.17,1.09,0.95,1.59,1.81,0.90,1.07,1.08,0.94,0.68,0.56,1.18,1.18,0.84,0.89,1.17,0.63,0.22,0.08,0.06,-0.04,-0.08,0.54,26,62.178333,0.802500,-203.363333,20.865000
3,PriceChange,4,100.00,71.88,68.75,59.38,62.50,62.50,56.25,68.75,59.38,62.50,62.50,62.50,59.38,59.38,56.25,65.62,62.50,59.38,65.62,50.00,56.25,56.25,46.88,50.00,0.75,0.86,0.80,0.57,0.77,1.09,0.51,0.91,0.59,0.78,0.56,0.52,0.68,0.54,0.64,0.81,0.93,0.63,0.32,0.09,0.18,0.22,-0.08,0.21,32,61.850000,0.578333,-260.800000,18.506667
1,PriceChange,2,100.00,71.43,71.43,61.90,66.67,61.90,57.14,61.90,52.38,57.14,57.14,52.38,52.38,57.14,47.62,57.14,52.38,47.62,57.14,47.62,47.62,47.62,42.86,47.62,0.63,1.02,1.12,0.96,2.00,2.33,1.28,1.09,0.65,0.92,0.60,0.33,0.45,0.22,-0.11,0.74,0.66,-0.11,0.12,-0.06,-0.12,-0.21,-0.11,-0.11,21,57.340417,0.595417,-265.851250,12.503750
19,PriceChange,20,91.27,71.43,69.05,59.52,61.90,62.70,65.87,69.05,70.63,67.46,67.46,67.46,66.67,65.08,66.67,69.05,69.84,71.43,72.22,66.67,66.67,65.87,61.90,62.70,0.41,0.30,0.28,0.20,0.36,0.44,0.51,0.58,0.57,0.50,0.56,0.59,0.62,0.64,0.72,0.84,1.06,0.98,0.79,0.70,0.77,0.72,0.78,0.83,126,67.857083,0.614583,-270.007500,77.437500
4,PriceChange,5,95.12,68.29,70.73,60.98,65.85,63.41,60.98,68.29,60.98,63.41,63.41,65.85,63.41,63.41,60.98,65.85,63.41,60.98,65.85,53.66,58.54,58.54,48.78,51.22,0.63,0.71,0.50,0.27,0.58,0.81,0.51,0.81,0.60,0.86,0.72,0.72,0.98,0.98,0.74,0.84,1.12,0.86,0.47,0.50,0.30,0.24,-0.06,0.52,41,63.413750,0.633750,-270.036250,25.983750
9,PriceChange,10,93.33,69.33,73.33,65.33,68.00,68.00,69.33,70.67,65.33,64.00,62.67,64.00,65.33,61.33,61.33,65.33,65.33,65.33,68.00,60.00,60.00,62.67,57.33,58.67,0.54,0.47,0.50,0.49,0.64,0.67,0.63,0.71,0.66,0.70,0.39,0.58,0.72,0.71,0.74,0.83,1.01,0.87,0.62,0.53,0.47,0.64,0.90,1.01,75,65.998750,0.667917,-300.093750,50.093750
13,PriceChange,14,94.44,68.89,72.22,65.56,64.44,65.56,68.89,71.11,66.67,66.67,64.44,64.44,65.56,61.11,63.33,66.67,66.67,66.67,68.89,62.22,60.00,62.22,56.67,57.78,0.52,0.30,0.48,0.41,0.44,0.54,0.59,0.69,0.66,0.62,0.49,0.58,0.62,0.53,0.61,0.79,0.88,0.86,0.61,0.53,0.47,0.58,0.57,0.83,90,66.296667,0.591667,-333.300000,53.250000
11,PriceChange,12,93.83,69.14,72.84,65.43,65.43,66.67,67.90,70.37,65.43,65.43,64.20,64.20,64.20,60.49,61.73,65.43,65.43,65.43,69.14,60.49,59.26,62.96,56.79,56.79,0.53,0.46,0.50,0.49,0.58,0.58,0.63,0.71,0.66,0.86,0.53,0.57,0.70,0.71,0.74,0.83,0.97,0.86,0.62,0.50,0.39,0.52,0.73,0.86,81,65.792083,0.647083,-340.841250,52.4

# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
1,48.911181,0.698576,-44.928403,0.749826,33
2,53.224514,1.224826,-96.531319,0.555521,60
4,63.002222,1.507500,-106.731771,3.426736,123
3,57.661771,1.482188,-108.683125,1.100035,90
5,63.067674,1.100035,-111.523819,7.445278,154
6,63.261736,1.080729,-129.090833,7.653299,176
7,59.737639,0.319965,-155.473542,7.016875,216
8,60.148194,0.825590,-160.278958,10.196632,242
9,60.661910,0.429826,-167.830035,11.735139,278


# Save new config data to config file

In [5]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [1]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)